# LousyBookML Unified Benchmark

This notebook benchmarks **PyTorch + TensorFlow + LousyBookML** using the **EXACT same dataset** for fair comparison.

## Instructions:
1. Run all cells (Runtime → Run all)
2. Download `colab_benchmark.json` when complete (contains BOTH data and results)
3. Place in your local `benchmark/` folder
4. Run `python benchmark/benchmark_runner.py` locally

## Step 1: Install Dependencies

In [ ]:
# Install PyTorch and TensorFlow
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu -q
!pip install tensorflow -q
print("[OK] PyTorch and TensorFlow installed")

## Step 2: Upload LousyBookML Package

Upload the `lousybookml` folder from your local machine:
1. Click the folder icon on the left sidebar
2. Click the upload button
3. Select your local `lousybookml` folder

Or run the cell below to install from the current directory if you've already uploaded it.

In [ ]:
# Add current directory to path for lousybookml
import sys
sys.path.insert(0, '/content')

# Try to import lousybookml
try:
    import lousybookml
    print("[OK] LousyBookML loaded")
except ImportError:
    print("[!] LousyBookML not found. Please upload the lousybookml folder.")
    print("    See instructions in the cell above.")

## Step 3: Run Unified Benchmark

In [ ]:
import numpy as np
import time
import json

# ==============================================================================
# DATA GENERATION (Fixed seed for reproducibility)
# ==============================================================================

def generate_and_save_data(n_samples=5000, n_features=20, noise=0.5, random_state=42):
    """Generate synthetic data with fixed seed and save to JSON."""
    np.random.seed(random_state)
    
    X = np.random.randn(n_samples, n_features)
    true_weights = np.random.randn(n_features) * 2
    true_bias = 5.0
    y = X @ true_weights + true_bias + np.random.randn(n_samples) * noise
    
    indices = np.random.permutation(n_samples)
    split_idx = int(0.8 * n_samples)
    train_idx, test_idx = indices[:split_idx], indices[split_idx:]
    
    X_train, X_test = X[train_idx].tolist(), X[test_idx].tolist()
    y_train, y_test = y[train_idx].tolist(), y[test_idx].tolist()
    
    data = {
        'n_samples': n_samples,
        'n_features': n_features,
        'noise': noise,
        'random_state': random_state,
        'true_weights': true_weights.tolist(),
        'true_bias': float(true_bias),
        'X_train': X_train,
        'X_test': X_test,
        'y_train': y_train,
        'y_test': y_test,
        'train_size': len(y_train),
        'test_size': len(y_test)
    }
    
    print(f"[OK] Generated benchmark data")
    print(f"     Samples: {n_samples}, Features: {n_features}")
    print(f"     Train: {len(y_train)}, Test: {len(y_test)}")
    
    return data

# Generate data
print("Generating dataset with fixed seed...")
data_info = generate_and_save_data()

# Load data as numpy arrays
X_train = np.array(data_info['X_train'])
X_test = np.array(data_info['X_test'])
y_train = np.array(data_info['y_train'])
y_test = np.array(data_info['y_test'])
n_features = data_info['n_features']

In [ ]:
# ==============================================================================
# PYTORCH BENCHMARK
# ==============================================================================

import torch
import torch.nn as nn
import torch.optim as optim

print("\n[PyTorch Benchmark]")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

pytorch_results = {}

# Convert to tensors
X_train_t = torch.FloatTensor(X_train).to(device)
y_train_t = torch.FloatTensor(y_train).to(device).view(-1, 1)
X_test_t = torch.FloatTensor(X_test).to(device)

# Method 1: torch.linalg.lstsq
print("Testing torch.linalg.lstsq...")
start = time.time()
X_train_bias = torch.cat([torch.ones(X_train_t.shape[0], 1).to(device), X_train_t], dim=1)
solution = torch.linalg.lstsq(X_train_bias, y_train_t).solution
fit_time = time.time() - start

start = time.time()
X_test_bias = torch.cat([torch.ones(X_test_t.shape[0], 1).to(device), X_test_t], dim=1)
y_pred = X_test_bias @ solution
predict_time = time.time() - start

y_pred_np = y_pred.cpu().numpy().flatten()
mse = np.mean((y_test - y_pred_np) ** 2)
r2 = 1 - np.sum((y_test - y_pred_np) ** 2) / np.sum((y_test - np.mean(y_test)) ** 2)

pytorch_results['PyTorch_LSTSQ'] = {
    'fit_time': fit_time,
    'predict_time': predict_time,
    'mse': float(mse),
    'rmse': float(np.sqrt(mse)),
    'mae': float(np.mean(np.abs(y_test - y_pred_np))),
    'r2': float(r2),
    'device': str(device)
}
print(f"  MSE: {mse:.6f}, R²: {r2:.6f}")

# Method 2: nn.Linear with SGD
print("Testing nn.Linear with SGD...")

class LinearModel(nn.Module):
    def __init__(self, n_features):
        super().__init__()
        self.linear = nn.Linear(n_features, 1)
    def forward(self, x):
        return self.linear(x)

model = LinearModel(n_features).to(device)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

start = time.time()
for epoch in range(1000):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_t)
    loss = criterion(outputs, y_train_t)
    loss.backward()
    optimizer.step()
fit_time = time.time() - start

start = time.time()
model.eval()
with torch.no_grad():
    y_pred = model(X_test_t)
predict_time = time.time() - start

y_pred_np = y_pred.cpu().numpy().flatten()
mse = np.mean((y_test - y_pred_np) ** 2)
r2 = 1 - np.sum((y_test - y_pred_np) ** 2) / np.sum((y_test - np.mean(y_test)) ** 2)

pytorch_results['PyTorch_SGD'] = {
    'fit_time': fit_time,
    'predict_time': predict_time,
    'mse': float(mse),
    'rmse': float(np.sqrt(mse)),
    'mae': float(np.mean(np.abs(y_test - y_pred_np))),
    'r2': float(r2),
    'epochs': 1000,
    'device': str(device)
}
print(f"  MSE: {mse:.6f}, R²: {r2:.6f}")
print("[OK] PyTorch benchmark complete")

In [ ]:
# ==============================================================================
# TENSORFLOW BENCHMARK
# ==============================================================================

import tensorflow as tf

print("\n[TensorFlow Benchmark]")
physical_devices = tf.config.list_physical_devices('GPU')
device_name = 'GPU' if len(physical_devices) > 0 else 'CPU'
print(f"Device: {device_name}")

tf_results = {}

# Convert to tensors
X_train_tf = tf.convert_to_tensor(X_train, dtype=tf.float32)
y_train_tf = tf.convert_to_tensor(y_train, dtype=tf.float32)
X_test_tf = tf.convert_to_tensor(X_test, dtype=tf.float32)

# Method 1: tf.linalg.lstsq
print("Testing tf.linalg.lstsq...")
start = time.time()
X_train_bias = tf.concat([tf.ones((X_train_tf.shape[0], 1)), X_train_tf], axis=1)
solution = tf.linalg.lstsq(X_train_bias, tf.expand_dims(y_train_tf, axis=-1))
fit_time = time.time() - start

start = time.time()
X_test_bias = tf.concat([tf.ones((X_test_tf.shape[0], 1)), X_test_tf], axis=1)
y_pred = tf.matmul(X_test_bias, solution)
predict_time = time.time() - start

y_pred_np = y_pred.numpy().flatten()
mse = np.mean((y_test - y_pred_np) ** 2)
r2 = 1 - np.sum((y_test - y_pred_np) ** 2) / np.sum((y_test - np.mean(y_test)) ** 2)

tf_results['TensorFlow_LSTSQ'] = {
    'fit_time': fit_time,
    'predict_time': predict_time,
    'mse': float(mse),
    'rmse': float(np.sqrt(mse)),
    'mae': float(np.mean(np.abs(y_test - y_pred_np))),
    'r2': float(r2),
    'device': device_name
}
print(f"  MSE: {mse:.6f}, R²: {r2:.6f}")

# Method 2: Keras Sequential
print("Testing Keras Sequential...")
start = time.time()
inputs = tf.keras.Input(shape=(n_features,))
outputs = tf.keras.layers.Dense(1, use_bias=True)(inputs)
model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.01), loss='mse')
history = model.fit(X_train_tf, y_train_tf, epochs=100, batch_size=32, verbose=0)
fit_time = time.time() - start

start = time.time()
y_pred = model.predict(X_test_tf, verbose=0)
predict_time = time.time() - start

y_pred_np = y_pred.flatten()
mse = np.mean((y_test - y_pred_np) ** 2)
r2 = 1 - np.sum((y_test - y_pred_np) ** 2) / np.sum((y_test - np.mean(y_test)) ** 2)

tf_results['TensorFlow_Keras'] = {
    'fit_time': fit_time,
    'predict_time': predict_time,
    'mse': float(mse),
    'rmse': float(np.sqrt(mse)),
    'mae': float(np.mean(np.abs(y_test - y_pred_np))),
    'r2': float(r2),
    'epochs': 100,
    'device': device_name
}
print(f"  MSE: {mse:.6f}, R²: {r2:.6f}")
print("[OK] TensorFlow benchmark complete")

In [ ]:
# ==============================================================================
# LOUSYBOOKML BENCHMARK (Optional - for same-environment comparison)
# ==============================================================================

try:
    from lousybookml.linear_model import LinearRegression, GradientDescentRegressor, Ridge
    from lousybookml import metrics
    
    print("\n[LousyBookML Benchmark]")
    lb_results = {}
    
    # OLS
    print("Testing LinearRegression...")
    start = time.time()
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    fit_time = time.time() - start
    
    start = time.time()
    y_pred = lr.predict(X_test)
    predict_time = time.time() - start
    
    lb_results['LousyBookML_OLS'] = {
        'fit_time': fit_time,
        'predict_time': predict_time,
        'mse': float(metrics.mse(y_test, y_pred)),
        'rmse': float(metrics.rmse(y_test, y_pred)),
        'mae': float(metrics.mae(y_test, y_pred)),
        'r2': float(metrics.r2_score(y_test, y_pred))
    }
    print(f"  MSE: {lb_results['LousyBookML_OLS']['mse']:.6f}, R²: {lb_results['LousyBookML_OLS']['r2']:.6f}")
    
    # Gradient Descent
    print("Testing GradientDescentRegressor...")
    start = time.time()
    gd = GradientDescentRegressor(learning_rate=0.01, n_iterations=1000)
    gd.fit(X_train, y_train)
    fit_time = time.time() - start
    
    start = time.time()
    y_pred = gd.predict(X_test)
    predict_time = time.time() - start
    
    lb_results['LousyBookML_GD'] = {
        'fit_time': fit_time,
        'predict_time': predict_time,
        'mse': float(metrics.mse(y_test, y_pred)),
        'rmse': float(metrics.rmse(y_test, y_pred)),
        'mae': float(metrics.mae(y_test, y_pred)),
        'r2': float(metrics.r2_score(y_test, y_pred)),
        'iterations': gd.n_iter_
    }
    print(f"  MSE: {lb_results['LousyBookML_GD']['mse']:.6f}, R²: {lb_results['LousyBookML_GD']['r2']:.6f}")
    
    # Ridge
    print("Testing Ridge...")
    start = time.time()
    ridge = Ridge(alpha=1.0)
    ridge.fit(X_train, y_train)
    fit_time = time.time() - start
    
    start = time.time()
    y_pred = ridge.predict(X_test)
    predict_time = time.time() - start
    
    lb_results['LousyBookML_Ridge'] = {
        'fit_time': fit_time,
        'predict_time': predict_time,
        'mse': float(metrics.mse(y_test, y_pred)),
        'rmse': float(metrics.rmse(y_test, y_pred)),
        'mae': float(metrics.mae(y_test, y_pred)),
        'r2': float(metrics.r2_score(y_test, y_pred))
    }
    print(f"  MSE: {lb_results['LousyBookML_Ridge']['mse']:.6f}, R²: {lb_results['LousyBookML_Ridge']['r2']:.6f}")
    print("[OK] LousyBookML benchmark complete")
    
except ImportError as e:
    print(f"[!] LousyBookML not available: {e}")
    lb_results = {}

In [ ]:
# ==============================================================================
# SAVE RESULTS
# ==============================================================================

# Combine all results
all_results = {}
all_results.update(pytorch_results)
all_results.update(tf_results)
if 'lb_results' in locals():
    all_results.update(lb_results)

# Create output
output = {
    'data_info': {
        'n_samples': data_info['n_samples'],
        'n_features': data_info['n_features'],
        'noise': data_info['noise'],
        'random_state': data_info['random_state'],
        'train_size': data_info['train_size'],
        'test_size': data_info['test_size']
    },
    'results': all_results,
    'timestamp': time.strftime('%Y-%m-%d %H:%M:%S'),
    'environment': 'Google Colab',
    'pytorch_version': str(torch.__version__),
    'tensorflow_version': str(tf.__version__)
}

# Save to SINGLE file (contains both data and results)
with open('colab_benchmark.json', 'w') as f:
    json.dump(output, f, indent=2)

print("\n" + "="*80)
print("BENCHMARK COMPLETE")
print("="*80)
print(f"\nTotal models benchmarked: {len(all_results)}")
print("\nFile to download:")
print("  colab_benchmark.json - contains BOTH data and results")
print("\nPlace in your local benchmark/ folder")
print("Then run: python benchmark/benchmark_runner.py")

# Display summary table
print("\n" + "="*80)
print("SUMMARY")
print("="*80)
print(f"{'Model':<30} {'Fit Time (s)':<15} {'MSE':<12} {'R²':<10}")
print("-"*80)
for model_name, result in all_results.items():
    fit_time = result.get('fit_time', 0)
    mse = result.get('mse', 0)
    r2 = result.get('r2', 0)
    print(f"{model_name:<30} {fit_time:<15.6f} {mse:<12.6f} {r2:<10.6f}")
print("="*80)

## Download Results

Run the cell below to download the single file, or click the folder icon on the left and download manually.

In [ ]:
# Download file automatically
from google.colab import files

print("Downloading colab_benchmark.json...")
files.download('colab_benchmark.json')

print("\n[OK] File downloaded!")